In [13]:
import numpy as np
from collections import defaultdict

import nbimporter
from HelperFunctions import SparseMatrix, buildLaplacian

In [14]:
def myLinearHarmonic(v: np.array, f: np.array, b: np.array, bc: np.array, k: int) -> np.array:
    """
    Inputs:
    v: Array of vertices
    f: Array of faces
    b: Array of indices for boundary vertices
    bc: Array of boundary values
    k: Power of harmonic operator
    ______________________________
    Output:
    w: Array of final vertices
    """
    
    #Step 1: Compute the (sparse) Laplacian Matrix
    L: SparseMatrix = buildLaplacian(v,f)

    #Step 2: Bring L to the correct harmonic degree
    A: SparseMatrix = L.pow_sparse(k)
   
    #Step 3: Solve the linear system
    
    #array of all vertices
    v_all: np.array = np.arange(v.shape[0])
    #array of non-boundary vertices
    v_in: np.array = np.setdiff1d(v_all,b)
    
    #slice it up
    A_ii: SparseMatrix = A.slice_sparse(v_in,v_in)
    A_ib: SparseMatrix = A.slice_sparse(v_in,b.flatten())

    #define B
    B: np.array = -1 * A_ib.sparse_times_dense(bc)

    #solve seperately for x,y,z
    w_x = SparseMatrix.solve_sparse(A_ii, B[:, 0])
    w_y = SparseMatrix.solve_sparse(A_ii, B[:, 1])
    w_z = SparseMatrix.solve_sparse(A_ii, B[:, 2])
    
    wi = np.stack((w_x, w_y, w_z), axis=1)
    
    #Step 4: we solved for interior so now concate with boundary vertices
    w: np.array = np.zeros_like(v)
    w[v_in] = wi
    w[b.flatten()] = bc
    
    return w
    